

# Homework 2 - Recurrent Neural Networks

In this part of the homework we are going to work with Recurrent Neural Networks, in particular GRU. One of the greatest things that Recurrent Neural Networks can do when working with sequences is retaining data from several timesteps in the past. We are going to explore that property by constructing an 'echo' Recurrent Neural Network.

The goal here is to make a model that given a sequence of letters or digits will output that same sequence, but with a certain delay. Let's say the input is a string 'abacaba', we want the model to not output anything for 3 steps (delay length), and then output the original string step by step, except the last 3 characters. So, target output is then 'XXXabac', where 'X' is empty output.

This is similar to [this notebook](https://github.com/Atcold/pytorch-Deep-Learning/blob/master/09-echo_data.ipynb) (which you should refer to when doing this assignment), except we're working not with a binary string, but with a sequence of integers between 0 and some N. In our case N is 26, which is the number of letters in the alphabet.

## Dataset

Let's implement the dataset. In our case, the data is basically infinite, as we can always generate more examples on the fly, so there's no need to load it from disk.

In [1]:
import random
import string

import torch

# Max value of the generated integer. 26 is chosen becuase it's
# the number of letters in English alphabet.
N = 26


def idx_to_onehot(x, k=N+1):
  """ Converts the generated integers to one-hot vectors """
  ones = torch.sparse.torch.eye(k)
  shape = x.shape
  res = ones.index_select(0, x.view(-1).type(torch.int64))
  return res.view(*shape, res.shape[-1])


class EchoDataset(torch.utils.data.IterableDataset):

  def __init__(self, delay=4, seq_length=15, size=1000):
    self.delay = delay
    self.seq_length = seq_length
    self.size = size

  def __len__(self):
    return self.size

  def __iter__(self):
    """ Iterable dataset doesn't have to implement __getitem__.
        Instead, we only need to implement __iter__ to return
        an iterator (or generator).
    """
    for _ in range(self.size):
      seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
      result = torch.cat((torch.zeros(self.delay), seq[:self.seq_length - self.delay])).type(torch.int64)
      yield seq, result

DELAY = 4
DATASET_SIZE = 200000
ds = EchoDataset(delay=DELAY, size=DATASET_SIZE)

## Model

Now, we want to implement the model. For our purposes, we want to use GRU. The architecture consists of GRU and a decoder. Decoder is responsible for decoding the GRU hidden state to yield a predicting for the next output. The parts you are responsible for filling with your code are marked with `TODO`.

In [7]:
class GRUMemory(torch.nn.Module):

  def __init__(self, hidden_size):
    super().__init__()
    #TODO: initialize your submodules
    self.gru = torch.nn.GRU(input_size=N + 1, hidden_size=hidden_size, batch_first=True)
    self.decoder = torch.nn.Linear(hidden_size, N + 1)

  def forward(self, x):
    # inputs: x - input tensor of shape (batch_size, seq_length, N+1)
    # returns:
    # logits (scores for softmax) of shape (batch size, seq_length, N + 1)
    # TODO implement forward pass
    batch_size, seq_length, _ = x.size()
    h_0 = torch.zeros(1, batch_size, self.gru.hidden_size).to(x.device)
    out, _ = self.gru(x, h_0)
    logits = self.decoder(out)
    return logits

  @torch.no_grad()
  def test_run(self, s):
    # This function accepts one string s containing lowercase characters a-z.
    # You need to map those characters to one-hot encodings,
    # then get the result from your network, and then convert the output
    # back to a string of the same length, with 0 mapped to ' ',
    # and 1-26 mapped to a-z.

    input_seq = torch.tensor([ord(char) - ord('a') + 1 for char in s], dtype=torch.int64)
    one_hot_input = idx_to_onehot(input_seq)
    logits = self.forward(one_hot_input.unsqueeze(0))
    _, predicted_indices = logits.max(dim=-1)
    predicted_chars = ''.join([chr(idx.item() + ord('a') - 1) if idx.item() > 0 else ' ' for idx in predicted_indices.squeeze()])

    return predicted_chars


## Training
Below you need to implement the training of the model. We give you more freedom as for the implementation. The two limitations are that it has to execute within 10 minutes, and that error rate should be below 1%.

In [8]:
def test_model(model, sequence_length=15):
    """
    This is the test function that runs 100 different strings through your model,
    and checks the error rate.
    """
    total = 0
    correct = 0
    for i in range(500):
        s = ''.join([random.choice(string.ascii_lowercase) for i in range(random.randint(15, 25))])
        result = model.test_run(s)
        for c1, c2 in zip(s[:-DELAY], result[DELAY:]):  # Note: Replace D with DELAY
            correct += int(c1 == c2)
        total += len(s) - DELAY

    return correct / total

In [4]:
import time
start_time = time.time()

# TODO
import torch.optim as optim

hidden_size = 64
model = GRUMemory(hidden_size)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

batch_size = 32
loader = torch.utils.data.DataLoader(ds, batch_size=batch_size, shuffle=False)

# Training loop
epochs = 5 
for epoch in range(epochs):
    total_loss = 0

    buffer = []
    for batch in loader:
        buffer.append(batch)
        if len(buffer) == batch_size:
            random.shuffle(buffer)

            for inputs, targets in buffer:
                optimizer.zero_grad()

                one_hot_inputs = idx_to_onehot(inputs)

                logits = model(one_hot_inputs)

                targets = targets.view(-1)

                loss = criterion(logits.view(-1, N + 1), targets)

                loss.backward()
                optimizer.step()

                total_loss += loss.item()

            buffer = []

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(loader)}")

accuracy = test_model(model)

end_time = time.time()
duration = end_time - start_time
accuracy = test_model(model)
assert duration < 600, 'execution took f{duration:.2f} seconds, which longer than 10 mins'
assert accuracy > 0.99, f'accuracy is too low, got {accuracy}, need 0.99'
print('tests passed')

Epoch 1/5, Loss: 0.4393750627301261
Epoch 2/5, Loss: 0.0006130304477410391
Epoch 3/5, Loss: 0.00017752305730959052
Epoch 4/5, Loss: 0.00014645102738482818
Epoch 5/5, Loss: 0.00010359139817766845
tests passed


## Variable delay model

Now, to make this more complicated, we want to have varialbe delay. So, now, the goal is to transform a sequence of pairs (character, delay) into a character sequence with given delay. Delay is constant within one sequence.

### Dataset
As before, we first implement the dataset:

In [12]:
class VariableDelayEchoDataset(torch.utils.data.IterableDataset):

  def __init__(self, max_delay=8, seq_length=20, size=1000):
    self.max_delay = max_delay
    self.seq_length = seq_length
    self.size = size

  def __len__(self):
    return self.size

  def __iter__(self):
    for _ in range(self.size):
      seq = torch.tensor([random.choice(range(1, N + 1)) for i in range(self.seq_length)], dtype=torch.int64)
      delay = random.randint(0, self.max_delay)
      result = torch.cat((torch.zeros(delay), seq[:self.seq_length - delay])).type(torch.int64)
      yield seq, delay, result

### Model

And the model.

In [29]:
class VariableDelayGRUMemory(torch.nn.Module):

  def __init__(self, hidden_size, max_delay):
    super().__init__()
    #TODO
    self.max_delay = max_delay
    self.gru = torch.nn.GRU(input_size=N + 1, hidden_size=hidden_size, batch_first=True)
    self.decoder = torch.nn.Linear(hidden_size, N + 1)

  def forward(self, x, delays):
      # inputs:
      # x - tensor of shape (batch size, seq length, N + 1)
      # delays - tensor of shape (batch size)
      # returns:
      # logits (scores for softmax) of shape (batch size, seq_length, N + 1)

      # TODO: Implement forward pass
      batch_size, seq_length, _ = x.size()
      h_0 = torch.zeros(1, batch_size, self.gru.hidden_size).to(x.device)

      out, _ = self.gru(x, h_0)

      delayed_hidden_states = torch.zeros(batch_size, seq_length, self.gru.hidden_size).to(x.device)

      for i in range(batch_size):
          delay = delays[i].item()
          for j in range(seq_length):
              if j < delay:
                  delayed_hidden_states[i, j] = out[i, j]
              else:
                  delayed_hidden_states[i, j] = out[i, j - delay]

      logits = self.decoder(delayed_hidden_states)
      return logits

  @torch.no_grad()
  def test_run(self, s, delay):
    # This function accepts one string s containing lowercase characters a-z,
    # and a delay - the desired output delay.
    # You need to map those characters to one-hot encodings,
    # then get the result from your network, and then convert the output
    # back to a string of the same length, with 0 mapped to ' ',
    # and 1-26 mapped to a-z.

    # TODO
    input_seq = torch.tensor([ord(char) - ord('a') + 1 for char in s], dtype=torch.int64)
    one_hot_input = idx_to_onehot(input_seq)
    delay_tensor = torch.tensor([delay], dtype=torch.int64)
    logits = self.forward(one_hot_input.unsqueeze(0), delay_tensor.unsqueeze(0))
    _, predicted_indices = logits.max(dim=-1)
    predicted_chars = ''.join([chr(idx.item() + ord('a') - 1) if idx.item() > 0 else ' ' for idx in predicted_indices.squeeze()])

    return predicted_chars


### Train

As before, you're free to do what you want, as long as training finishes within 10 minutes and accuracy is above 0.99 for delays between 0 and 8.

In [30]:
def test_variable_delay_model(model, seq_length=20):
  """
  This is the test function that runs 100 different strings through your model,
  and checks the error rate.
  """
  total = 0
  correct = 0
  for i in range(500):
    s = ''.join([random.choice(string.ascii_lowercase) for i in range(seq_length)])
    d = random.randint(0, model.max_delay)
    result = model.test_run(s, d)
    if d > 0:
      z = zip(s[:-d], result[d:])
    else:
      z = zip(s, result)
    for c1, c2 in z:
      correct += int(c1 == c2)
    total += len(s) - d

  return correct / total

In [46]:
import time
start_time = time.time()

MAX_DELAY = 8
SEQ_LENGTH = 20

# TODO: implement model training here.
hidden_size = 256
model = VariableDelayGRUMemory(hidden_size, MAX_DELAY)
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

batch_size = 32
loader = torch.utils.data.DataLoader(VariableDelayEchoDataset(max_delay=MAX_DELAY, size=1000), batch_size=batch_size)

# Training loop
epochs = 10 
for epoch in range(epochs):
    total_loss = 0

    buffer = []
    for batch in loader:
        buffer.append(batch)
        if len(buffer) == batch_size:
            for inputs, delays, targets in buffer:
                optimizer.zero_grad()

                one_hot_inputs = idx_to_onehot(inputs)

                logits = model(one_hot_inputs, delays)

                targets = targets.view(-1)

                logits = logits.view(-1, logits.size(-1))[:targets.size(0), :]

                loss = criterion(logits, targets[:logits.size(0)])

                loss.backward()
                optimizer.step()

                total_loss += loss.item()

            buffer = []

    print(f"Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(loader)}")

# Evaluate the model
accuracy = test_variable_delay_model(model)

end_time = time.time()
assert end_time - start_time < 600, 'executing took longer than 10 mins'
assert test_variable_delay_model(model) > 0.99, 'accuracy is too low'
print('tests passed')

Epoch 1/10, Loss: 3.1119643598794937
Epoch 2/10, Loss: 2.5204399451613426
Epoch 3/10, Loss: 1.609206460416317
Epoch 4/10, Loss: 0.8000087775290012
Epoch 5/10, Loss: 0.5392754031345248
Epoch 6/10, Loss: 0.4799939878284931
Epoch 7/10, Loss: 0.43709477595984936
Epoch 8/10, Loss: 0.43238019570708275
Epoch 9/10, Loss: 0.4061380662024021
Epoch 10/10, Loss: 0.3926347102969885
tests passed
